In [ ]:
import kagglehub
import shutil
import os
# Download latest version
path = kagglehub.dataset_download("kaggleashwin/vehicle-type-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/vehicle-type-recognition


In [ ]:
local_path = "./CarMotor_dataset/"

# Ensure the local directory exists
os.makedirs(local_path, exist_ok=True)
# Copy the dataset to the local directory
for item in os.listdir(path):
    s = os.path.join(path, item)
    d = os.path.join(local_path, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)

print(f"Dataset saved at: {local_path}")

print(f"Dataset saved at: {local_path}")

Dataset saved at: ./CarMotor_dataset/
Dataset saved at: ./CarMotor_dataset/


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
import os

folder_path = 'CarMotor_dataset/Dataset/Car'

# Count only image files
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp')
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Number of image files: {num_images}")


Number of image files: 100


In [ ]:
import os

folder_path = 'CarMotor_dataset/Dataset/motorcycle'

# Count only image files
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp')
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Number of image files: {num_images}")


Number of image files: 100


In [ ]:
import os, random, shutil

# Set up paths
source_path = "CarMotor_dataset/Dataset"
train_path = "CarMotor_dataset/train"
test_path = "CarMotor_dataset/test"
classes = ["Car", "motorcycle"]

# Make train and test folders
for cls in classes:
    os.makedirs(os.path.join(train_path, cls), exist_ok=True)
    os.makedirs(os.path.join(test_path, cls), exist_ok=True)

# Move 80% of images to train, 20% to test
for cls in classes:
    files = os.listdir(os.path.join(source_path, cls))
    random.shuffle(files)

    split = int(0.85 * len(files))
    train_files = files[:split]
    test_files = files[split:]

    for f in train_files:
        shutil.move(os.path.join(source_path, cls, f), os.path.join(train_path, cls, f))
    for f in test_files:
        shutil.move(os.path.join(source_path, cls, f), os.path.join(test_path, cls, f))

print("✅ Images moved into train/ and test/ folders.")


✅ Images moved into train/ and test/ folders.


In [ ]:
train_dir = "/content/CarMotor_dataset/train"
test_dir = "/content/CarMotor_dataset/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 200 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (Car vs Motorcycle)
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator
)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.5913 - loss: 0.6693 - val_accuracy: 0.6905 - val_loss: 0.5964
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7202 - loss: 0.5886 - val_accuracy: 0.6845 - val_loss: 0.6828
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.6817 - loss: 0.5525 - val_accuracy: 0.5000 - val_loss: 1.0061
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.6647 - loss: 0.6122 - val_accuracy: 0.7143 - val_loss: 0.4842
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7951 - loss: 0.4683 - val_accuracy: 0.8631 - val_loss: 0.3272
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7746 - loss: 0.4316 - val_accuracy: 0.8452 - val_loss: 0.3312
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.8362 - loss: 0.3742 - val_accuracy: 0.7798 - val_loss: 0.4347
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.8441 - loss: 0.3260 - val_accuracy: 0.8214 - val_loss: 0.4716
Epoch 9/

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")


In [ ]:
print(train_generator.class_indices)



In [ ]:
train_generator.classes


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    prediction = model.predict(img_array)
    label = "Motorcycle" if prediction[0][0] > 0.5 else "Car"

    # Display the image
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Prediction: {label}", fontsize=14, fontweight='bold')
    plt.show()

In [ ]:
img_path ="/content/CarMotor_dataset/test/Car/Image_25.jpg"
print(predict_image(img_path))

In [ ]:
img_path ="/content/CarMotor_dataset/test/motorcycle/Image_20.jpg"
print(predict_image(img_path))

In [ ]:
img_path ="/content/CarMotor_dataset/test/Car/Image_9.jpg"
print(predict_image(img_path))

In [ ]:
img_path="/content/CarMotor_dataset/test/motorcycle/Image_45.jpg"
print(predict_image(img_path))

In [ ]:
img_path ="/content/Moto 2.png"
print(predict_image(img_path)) #test with real life photos